In [1]:
import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

C:\Users\ShaoDan\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
# Read in data
df = pd.read_excel("correlation.xlsx", sheet_name='dataselect', parse_dates=True, index_col="Dates")

In [6]:
df.head()

,FNERTR Index,CRSPSCT Index,CRSPLC1 Index,XAU Curncy,CL1 Comdty,LF98TRUU Index,LUATTRUU Index
Dates,,,,,,,
2011-03-31 00:00:00,8973.82,1000,1000,1432.2,106.72,1258.19,1840.37
2011-04-01 00:00:00,8991.27,1005.24,1005.09,1428.73,107.94,1260.63,1840.16
2011-04-04 00:00:00,8987.19,1007.19,1005.49,1434.03,108.47,1262.92,1842.92
2011-04-05 00:00:00,8987.67,1013.35,1005.28,1455.85,108.34,1263.58,1837.95
2011-04-06 00:00:00,8984.33,1014.59,1006.92,1459.75,108.83,1265.45,1833.11


In [5]:
df.tail()

,FNERTR Index,CRSPSCT Index,CRSPLC1 Index,XAU Curncy,CL1 Comdty,LF98TRUU Index,LUATTRUU Index
Dates,,,,,,,
2019-03-14 00:00:00,19103.5,2210.74,2106.75,1296.17,58.61,2033.79,2232.84
2019-03-15 00:00:00,19028.9,2214.88,2116.99,1302.4,58.52,2035.21,2236.97
2019-03-18 00:00:00,18928.9,2228.16,2124.79,1303.71,59.09,2036.45,2236.95
2019-03-19 00:00:00,18873,2217.3,2124.41,1306.56,59.03,2038.17,2235.21
2019-03-20 00:00:00,18873,2217.3,2124.41,1303,59.01,2038.17,2235.21


In [4]:
# 清理列
drop_volumns = list(filter(lambda c: c.startswith('LOG'),df.columns.values.tolist()))
df = df.drop(columns=drop_volumns)
# 清理行
df = df[df.index.notnull()]
df = df.drop('variance')
# 没有REITs
df_without_reits = df.drop(columns=['FNERTR Index'])

In [7]:
# Calculate expected returns and sample covariance
def efficient_frontier(df):
    mu = expected_returns.mean_historical_return(df)
#     S = risk_models.sample_cov(df)
    S = risk_models.CovarianceShrinkage(df).ledoit_wolf()

    # Optimise for maximal Sharpe ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)

In [8]:
efficient_frontier(df)

{'FNERTR Index': 0.00404, 'CRSPSCT Index': 0.00761, 'CRSPLC1 Index': 0.05923, 'XAU Curncy': 0.0, 'CL1 Comdty': 0.0, 'LF98TRUU Index': 0.63172, 'LUATTRUU Index': 0.2974}
Expected annual return: 5.2%
Annual volatility: 2.9%
Sharpe Ratio: 1.07


In [9]:
efficient_frontier(df_without_reits)

{'CRSPSCT Index': 0.00892, 'CRSPLC1 Index': 0.05944, 'XAU Curncy': 0.0, 'CL1 Comdty': 0.0, 'LF98TRUU Index': 0.62968, 'LUATTRUU Index': 0.30196}
Expected annual return: 5.2%
Annual volatility: 2.9%
Sharpe Ratio: 1.08


In [55]:
df.to_csv("reits_clean.csv")